In [7]:
from mongoengine import connect
import math
import json
import sys
sys.path.insert(0, '..\\mongo-db-scripts')
from Bug import Bug

connect('bug_reports_db', host='127.0.0.1', port=27017)

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [7]:
def count_unique_users_and_modifications(history):
  unique_users = 0
  total_modifications = 0
  users = []

  for modifications in history:
    total_modifications += 1

    current_user = modifications['who']

    if current_user not in users:
      unique_users += 1
      users.append(current_user)
      
  return { 'unique_users': unique_users, 'modifications': total_modifications }

In [ ]:
BUGS_LIMIT = 50000
total_bugs = Bug.objects.count()
bugs_per_iteration = math.ceil(total_bugs/BUGS_LIMIT)

bugs_unique_users = []

for i in range(bugs_per_iteration):
  skip_quantity = i * BUGS_LIMIT
  result = Bug.objects.only('bug_id', 'history').limit(BUGS_LIMIT).skip(skip_quantity)

  bugs = [bug.to_mongo().to_dict() for bug in result]

  for bug in bugs:
    temp_bug = count_unique_users_and_modifications(bug['history'])
    temp_bug['bug_id'] = bug['bug_id']
    bugs_unique_users.append(temp_bug)
                    

In [10]:
with open('data\\processed_bugs_changes_unique_users.json', 'w') as output:
  json.dump(bugs_unique_users, output)